# Unit 3.2 – Deployment and Monitoring with Lightweight CI/CD (MLOps-lite)
## Jupyter Lab Notebook

This notebook simulates a lightweight CI/CD workflow for Native AI systems without Docker or external CI platforms.
We implement pipeline logic step-by-step in Python so that quality gates, artifact packaging, monitoring signals, and rollback rules remain transparent and reproducible.


In [1]:
import os
import json
from datetime import datetime
from pathlib import Path

import numpy as np

# Project root = parent of notebooks/
ROOT = Path.cwd().parent

SRC_DIR = ROOT / "src"
ARTIFACTS_DIR = ROOT / "artifacts"
REPORTS_DIR = ARTIFACTS_DIR / "reports"
RELEASES_DIR = ARTIFACTS_DIR / "releases"

for p in [SRC_DIR, REPORTS_DIR, RELEASES_DIR]:
    p.mkdir(parents=True, exist_ok=True)

print("ROOT:", ROOT)
print("SRC_DIR:", SRC_DIR)
print("REPORTS_DIR:", REPORTS_DIR)
print("RELEASES_DIR:", RELEASES_DIR)


ROOT: C:\Users\ikybe\5g-digits\unit32
SRC_DIR: C:\Users\ikybe\5g-digits\unit32\src
REPORTS_DIR: C:\Users\ikybe\5g-digits\unit32\artifacts\reports
RELEASES_DIR: C:\Users\ikybe\5g-digits\unit32\artifacts\releases


## Lab Step 1: Implement Quality Gates from Benchmarks

Goal: Convert benchmarking results into automated **pass/fail** decisions (quality gates).

In a real CI pipeline, this step would run automatically on every update.
Here, we simulate it locally and generate:
- a KPI report (JSON)
- a gate decision (PASS/FAIL)
- a short human-readable summary

We will use *illustrative KPIs* that align with edge AI constraints:
- Accuracy (higher is better)
- Latency (lower is better)
- Model size (lower is better)
- Validation failure rate (lower is better)


In [2]:
import sys
sys.path.insert(0, str(ROOT))  # allow "from src..." imports

from src.gates import GateThresholds, evaluate_gates
from src.utils_io import timestamp_id, write_json

# Synthetic KPIs (we will later replace with actual measurements if desired)
kpis = {
    "accuracy": 0.88,
    "latency_ms": 22.4,
    "model_size_mb": 4.2,
    "validation_fail_rate": 0.01
}

thresholds = GateThresholds(
    min_accuracy=0.85,
    max_latency_ms=25.0,
    max_model_size_mb=5.0,
    max_validation_fail_rate=0.02
)

overall_pass, gate_details = evaluate_gates(kpis, thresholds)

run_id = timestamp_id()
report = {
    "run_id": run_id,
    "kpis": kpis,
    "thresholds": thresholds.__dict__,
    "gate_details": gate_details
}

report_path = REPORTS_DIR / f"gate_report_{run_id}.json"
write_json(report_path, report)

print("Gate decision:", "PASS ✅" if overall_pass else "FAIL ❌")
print("Report saved to:", report_path)


Gate decision: PASS ✅
Report saved to: C:\Users\ikybe\5g-digits\unit32\artifacts\reports\gate_report_20251215_194241.json


Print a readable summary

In [3]:
checks = gate_details["checks"]

print("\nQuality Gate Summary")
print("-" * 60)
for name, info in checks.items():
    status = "PASS" if info["pass"] else "FAIL"
    print(f"{name:22s} {status:4s} | value={info['value']} | threshold={info['threshold']} | {info['direction']}")

print("-" * 60)
print("Overall:", "PASS ✅" if gate_details["overall_pass"] else "FAIL ❌")



Quality Gate Summary
------------------------------------------------------------
accuracy               PASS | value=0.88 | threshold=0.85 | higher_is_better
latency_ms             PASS | value=22.4 | threshold=25.0 | lower_is_better
model_size_mb          PASS | value=4.2 | threshold=5.0 | lower_is_better
validation_fail_rate   PASS | value=0.01 | threshold=0.02 | lower_is_better
------------------------------------------------------------
Overall: PASS ✅


## Lab Step 2: Package a Release Artifact (MLOps-lite)

Goal: create a simple, reproducible “release bundle” that could be deployed on an edge device.

In real CI/CD, a successful gate decision would automatically trigger packaging.  
Here, we simulate this by building a release folder containing:

- `manifest.json` (metadata: run_id, KPIs, gate result, versions)
- `gate_report_*.json` (the full gate report)
- `README.txt` (human-friendly deployment notes)

This step does **not** require Docker or a real deployment platform; it focuses on the core idea:
**only gated builds become deployable artifacts.**


In [6]:
# 2.1 - Locate latest gate report and load it (with safe reload of src.utils_io)

import json
import importlib
import src.utils_io as utils_io

# Reload to ensure Jupyter picks up your latest edits
importlib.reload(utils_io)

write_text = utils_io.write_text  # now guaranteed to exist if it's in the file

# Find the most recent gate report in artifacts/reports
reports = sorted(REPORTS_DIR.glob("gate_report_*.json"), reverse=True)
assert len(reports) > 0, f"No gate reports found in {REPORTS_DIR}"

latest_report_path = reports[0]
with open(latest_report_path, "r", encoding="utf-8") as f:
    latest_report = json.load(f)

run_id = latest_report["run_id"]
overall_pass = latest_report["gate_details"]["overall_pass"]

print("Latest report:", latest_report_path.name)
print("run_id:", run_id)
print("Gate decision:", "PASS ✅" if overall_pass else "FAIL ❌")

# 2.2. Quick sanity check (should be 'True')
print("write_text available:", hasattr(utils_io, "write_text"))


Latest report: gate_report_20251215_194241.json
run_id: 20251215_194241
Gate decision: PASS ✅
write_text available: True


### Packaging rule

We only create a release artifact when the gate decision is **PASS**.
If the decision is **FAIL**, the pipeline stops here (no deployable bundle is produced).


In [7]:
# 2.3 -  Release folder (one per run_id)
release_dir = RELEASES_DIR / f"release_{run_id}"

if not overall_pass:
    print("Gate decision is FAIL ❌ — skipping packaging.")
else:
    release_dir.mkdir(parents=True, exist_ok=True)

    # 1) Copy the gate report into the release bundle
    gate_copy_path = release_dir / latest_report_path.name
    gate_copy_path.write_text(json.dumps(latest_report, indent=2, sort_keys=True), encoding="utf-8")

    # 2) Create a short manifest.json (minimal deployment metadata)
    manifest = {
        "run_id": run_id,
        "created_utc": datetime.utcnow().isoformat() + "Z",
        "gate_decision": "PASS",
        "kpis": latest_report["kpis"],
        "thresholds": latest_report["thresholds"],
        "notes": "Release created by Unit 3.2 notebook pipeline (MLOps-lite)."
    }
    manifest_path = release_dir / "manifest.json"
    manifest_path.write_text(json.dumps(manifest, indent=2, sort_keys=True), encoding="utf-8")

    # 3) Create a human-readable README
    readme = f"""Unit 3.2 – Release Artifact

run_id: {run_id}
gate_decision: PASS

Included files:
- {latest_report_path.name} : full quality gate report
- manifest.json             : compact metadata for deployment tooling
- README.txt                : this file

Deployment note:
This bundle is a simulation of a CI/CD release artifact. In a real system, it would also include
model binaries, configuration, and versioned dependencies.
"""
    readme_path = release_dir / "README.txt"
    write_text(readme_path, readme)

    print("Release bundle created ✅")
    print("Release directory:", release_dir)
    print("Files:", [p.name for p in sorted(release_dir.iterdir())])


Release bundle created ✅
Release directory: C:\Users\ikybe\5g-digits\unit32\artifacts\releases\release_20251215_194241
Files: ['gate_report_20251215_194241.json', 'manifest.json', 'README.txt']


In [9]:
# 2.4 - Sanity check: read manifest back
if overall_pass:
    with open(release_dir / "manifest.json", "r", encoding="utf-8") as f:
        m = json.load(f)

    print("\nManifest check")
    print("-" * 60)
    print("run_id:", m["run_id"])
    print("gate_decision:", m["gate_decision"])
    print("kpis:", m["kpis"])
    print("-" * 60)
else:
    print("No manifest created because packaging was skipped (FAIL gate).")



Manifest check
------------------------------------------------------------
run_id: 20251215_194241
gate_decision: PASS
kpis: {'accuracy': 0.88, 'latency_ms': 22.4, 'model_size_mb': 4.2, 'validation_fail_rate': 0.01}
------------------------------------------------------------


### Optional: Human-Readable Release Summary 

Some release manifests may not explicitly include a `gate_report` field.
To keep this step robust, we:

1. Read `manifest.json`
2. Try to load the gate report referenced by the manifest
3. If missing, auto-detect `gate_report_*.json` inside the release directory


In [12]:
# 2.5 human-redable summary
import json
from pathlib import Path

print("\nRelease Summary")
print("-" * 60)
print("Release dir:", release_dir)

manifest_path = release_dir / "manifest.json"
assert manifest_path.exists(), f"manifest.json not found in {release_dir}"

with open(manifest_path, "r", encoding="utf-8") as f:
    manifest = json.load(f)

print("run_id:", manifest.get("run_id"))
print("gate_decision:", manifest.get("gate_decision"))
print("kpis:", manifest.get("kpis", {}))

# --- Resolve gate report path (preferred: manifest pointer; fallback: autodetect) ---
gate_report_path = None

gate_report_name = manifest.get("gate_report")
if isinstance(gate_report_name, str) and gate_report_name.strip():
    candidate = release_dir / gate_report_name
    if candidate.exists():
        gate_report_path = candidate

if gate_report_path is None:
    # Fallback: locate gate_report_*.json in the release folder
    candidates = sorted(release_dir.glob("gate_report_*.json"), reverse=True)
    if len(candidates) > 0:
        gate_report_path = candidates[0]

if gate_report_path is None:
    print("\nNo gate report found in release folder.")
else:
    print("\nGate report:", gate_report_path.name)

    with open(gate_report_path, "r", encoding="utf-8") as f:
        gate_report = json.load(f)

    gate_details = gate_report.get("gate_details", {})
    checks = gate_details.get("checks", {})

    print("\nChecks:")
    for name, info in checks.items():
        status = "PASS" if info.get("pass") else "FAIL"
        value = info.get("value")
        threshold = info.get("threshold")
        direction = info.get("direction")
        print(f"  - {name}: {status} | value={value} | threshold={threshold} | {direction}")

    print("\nOverall:", "PASS ✅" if gate_details.get("overall_pass") else "FAIL ❌")




Release Summary
------------------------------------------------------------
Release dir: C:\Users\ikybe\5g-digits\unit32\artifacts\releases\release_20251215_194241
run_id: 20251215_194241
gate_decision: PASS
kpis: {'accuracy': 0.88, 'latency_ms': 22.4, 'model_size_mb': 4.2, 'validation_fail_rate': 0.01}

Gate report: gate_report_20251215_194241.json

Checks:
  - accuracy: PASS | value=0.88 | threshold=0.85 | higher_is_better
  - latency_ms: PASS | value=22.4 | threshold=25.0 | lower_is_better
  - model_size_mb: PASS | value=4.2 | threshold=5.0 | lower_is_better
  - validation_fail_rate: PASS | value=0.01 | threshold=0.02 | lower_is_better

Overall: PASS ✅


## LAB STEP 3: Deployment Dry-Run (No Docker)

In the real pipeline, a passing **quality gate** would trigger an automated deployment step.

In this lab, we **simulate** that deployment without Docker or real infrastructure by:
1. Creating a `deployment_plan.json` from the release manifest
2. Running a “preflight” validation (files exist, metadata present)
3. Producing a lightweight `deployment_receipt.json` that represents an executed deployment


In [13]:
# 3.1 Create a deployment plan
import json
from pathlib import Path
from datetime import datetime

DEPLOYMENTS_DIR = ARTIFACTS_DIR / "deployments"
DEPLOYMENTS_DIR.mkdir(parents=True, exist_ok=True)

# Inputs from previous steps:
# - release_dir (Path)
# - manifest (dict) OR manifest_path
manifest_path = release_dir / "manifest.json"
assert manifest_path.exists(), f"manifest.json not found: {manifest_path}"

with open(manifest_path, "r", encoding="utf-8") as f:
    manifest = json.load(f)

run_id = manifest.get("run_id") or release_dir.name.replace("release_", "")
deployment_id = f"deploy_{run_id}"
deployment_dir = DEPLOYMENTS_DIR / deployment_id
deployment_dir.mkdir(parents=True, exist_ok=True)

plan = {
    "deployment_id": deployment_id,
    "created_at": datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ"),
    "source_release_dir": str(release_dir),
    "run_id": run_id,
    "gate_decision": manifest.get("gate_decision"),
    "kpis": manifest.get("kpis", {}),
    "artifacts": {
        "manifest": "manifest.json",
        # optional field; may not exist in your manifest, but we keep it for completeness
        "gate_report": manifest.get("gate_report", None),
    },
    "target": {
        "environment": "staging",
        "channel": "simulated",
        "rollout": "100%",
    },
}

plan_path = deployment_dir / "deployment_plan.json"
with open(plan_path, "w", encoding="utf-8") as f:
    json.dump(plan, f, indent=2, sort_keys=True)

print("Deployment plan created ✅")
print("Plan saved to:", plan_path)


Deployment plan created ✅
Plan saved to: C:\Users\ikybe\5g-digits\unit32\artifacts\deployments\deploy_20251215_194241\deployment_plan.json


In [14]:
# 3.2 Preflight checks
import os

def preflight_check(release_dir: Path) -> dict:
    required = ["manifest.json", "README.txt"]
    present = {name: (release_dir / name).exists() for name in required}

    # Gate report is optional (can be discovered later), so we don't fail if missing
    gate_reports = sorted(release_dir.glob("gate_report_*.json"), reverse=True)
    present["gate_report_autodetect"] = len(gate_reports) > 0

    ok = all(present[name] for name in required)
    return {
        "ok": ok,
        "present": present,
        "autodetected_gate_report": gate_reports[0].name if gate_reports else None
    }

preflight = preflight_check(release_dir)

print("\nPreflight check")
print("-" * 60)
for k, v in preflight["present"].items():
    print(f"{k:24s}: {v}")

print("-" * 60)
print("Preflight:", "PASS ✅" if preflight["ok"] else "FAIL ❌")
print("Gate report autodetect:", preflight["autodetected_gate_report"])



Preflight check
------------------------------------------------------------
manifest.json           : True
README.txt              : True
gate_report_autodetect  : True
------------------------------------------------------------
Preflight: PASS ✅
Gate report autodetect: gate_report_20251215_194241.json


In [15]:
# 3.3 Simulated deployment
import json
from datetime import datetime

# Load plan back
with open(plan_path, "r", encoding="utf-8") as f:
    plan_loaded = json.load(f)

# Only deploy if gate passed AND preflight passed
gate_ok = (str(plan_loaded.get("gate_decision")).upper() == "PASS")
preflight_ok = bool(preflight.get("ok"))

deployed = gate_ok and preflight_ok

receipt = {
    "deployment_id": plan_loaded["deployment_id"],
    "executed_at": datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ"),
    "status": "DEPLOYED" if deployed else "SKIPPED",
    "reason": None if deployed else {
        "gate_ok": gate_ok,
        "preflight_ok": preflight_ok
    },
    "source_release_dir": plan_loaded["source_release_dir"],
    "target": plan_loaded["target"],
}

receipt_path = deployment_dir / "deployment_receipt.json"
with open(receipt_path, "w", encoding="utf-8") as f:
    json.dump(receipt, f, indent=2, sort_keys=True)

print("Deployment result:", receipt["status"], ("✅" if deployed else "⚠️"))
print("Receipt saved to:", receipt_path)


Deployment result: DEPLOYED ✅
Receipt saved to: C:\Users\ikybe\5g-digits\unit32\artifacts\deployments\deploy_20251215_194241\deployment_receipt.json


### 3.4 Post-deploy smoke check (simulated)

After a deployment, teams usually run a small “smoke test” to confirm:
- the deployment receipt exists and indicates `DEPLOYED`
- the referenced release directory still exists
- the manifest can be loaded and still reports a PASS gate decision

This is not a functional test of the full system — it is a lightweight integrity check.


In [16]:
# 3.4 Post-deploy smoke check
import json
from pathlib import Path

# We assume these variables exist from prior cells:
# - deployment_dir (Path)
# - release_dir (Path)

receipt_path = deployment_dir / "deployment_receipt.json"
assert receipt_path.exists(), f"Missing receipt: {receipt_path}"

with open(receipt_path, "r", encoding="utf-8") as f:
    receipt = json.load(f)

print("Smoke check")
print("-" * 60)
print("deployment_id:", receipt.get("deployment_id"))
print("status:", receipt.get("status"))

assert receipt.get("status") == "DEPLOYED", "Smoke check failed: deployment was not DEPLOYED"
assert Path(receipt.get("source_release_dir")).exists(), "Smoke check failed: source release dir missing"

manifest_path = release_dir / "manifest.json"
assert manifest_path.exists(), f"Missing manifest: {manifest_path}"

with open(manifest_path, "r", encoding="utf-8") as f:
    manifest = json.load(f)

print("gate_decision (manifest):", manifest.get("gate_decision"))
assert str(manifest.get("gate_decision")).upper() == "PASS", "Smoke check failed: gate_decision is not PASS"

print("-" * 60)
print("Smoke check: PASS ✅")


Smoke check
------------------------------------------------------------
deployment_id: deploy_20251215_194241
status: DEPLOYED
gate_decision (manifest): PASS
------------------------------------------------------------
Smoke check: PASS ✅


### 3.5 Optional rollback simulation (simulated)

If a smoke test fails in production, the common response is to:
- mark the deployment as rolled back
- preserve the original receipt for auditability

Here we simulate a rollback by writing a `rollback_receipt.json`.


In [17]:
# 3.5 Optional rollback simulation
import json
from datetime import datetime

rollback = {
    "deployment_id": receipt.get("deployment_id"),
    "rolled_back_at": datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ"),
    "status": "ROLLED_BACK",
    "reason": "simulated_rollback_for_training",
    "previous_status": receipt.get("status"),
}

rollback_path = deployment_dir / "rollback_receipt.json"
with open(rollback_path, "w", encoding="utf-8") as f:
    json.dump(rollback, f, indent=2, sort_keys=True)

print("Rollback receipt written ⚠️")
print("Path:", rollback_path)


Rollback receipt written ⚠️
Path: C:\Users\ikybe\5g-digits\unit32\artifacts\deployments\deploy_20251215_194241\rollback_receipt.json
